<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Linear regression

In this exercise you will use linear regression to predict flat prices. Training will be handled via gradient descent and we will:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a non-standard loss function.

Let's start with getting the data.

In [1]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2023-10-18 18:18:10--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/zey0gx91pna8irj/mieszkania.csv [following]
--2023-10-18 18:18:10--  https://www.dropbox.com/s/dl/zey0gx91pna8irj/mieszkania.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc36886bcaccbcc95736b3ac210b.dl.dropboxusercontent.com/cd/0/get/CF3Wd-ShF190VInvEhZ4vkK9q-91r6P1S_RN4UrDmZonViLWbx4dtm0bmIiq1p7xWbHV8A-tCPqvJTJjkH-al0an7FdmdKCiN7OLZhJgjRmzem-_KizpLiJNc04WM6DDhHU/file?dl=1# [following]
--2023-10-18 18:18:10--  https://uc36886bcaccbcc95736b3ac210b.dl.dropboxusercontent.com/cd/0/get/CF3Wd-ShF190VInvEhZ4vkK9q-91r6P1S_RN4UrDmZonViLWbx4dtm0bmIiq1p7xWbHV8A-tCPqvJTJjkH-al0an7FdmdKCiN7OLZhJgjRmzem-_Kiz

In [2]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilosc_sypialni,ilosc_lazienek,rok_budowy,parking_podziemny`).

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

Let's start with implementing the loss function.

In [3]:
import numpy as np

def msle(ys, ps):
    assert len(ys) == len(ps)
    #################################
    # TODO: Implement this function #
    #################################
    return np.mean((np.log(1+np.array(ys))-np.log(1+np.array(ps)))**2)

ys = 1, 1, 0
ps1 = 0.1, 0.1, 0.1
ps2 = 0.9, 0.9, 0.1

print(ys, ps1, msle(ys, ps1))
print(ys, ps2, msle(ys, ps2))

(1, 1, 0) (0.1, 0.1, 0.1) 0.2413007297730947
(1, 1, 0) (0.9, 0.9, 0.1) 0.0047820114908628695


The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [4]:
###################################################
# TODO: Compute msle for outputing the mean price #
###################################################
import pandas as pd

mieszkania = pd.read_csv('mieszkania.csv')
const = np.mean(mieszkania['cena'])
print(f"Log-square loss for constant-mean pred: {msle(mieszkania['cena'], [const]*len(mieszkania))}")

Log-square loss for constant-mean pred: 0.3915253538257009


Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

In [5]:
#############################################
# TODO: Find this constant and compute msle #
#############################################
const = np.exp(np.log(mieszkania['cena']).mean())-1
print(f"Log-square loss for constant-mean pred: {msle(mieszkania['cena'], [const]*len(mieszkania))}")

Log-square loss for constant-mean pred: 0.36488961222131766


Now, let's implement a standard linear regression model.

In [6]:
##########################################################
# TODO: Implement linear regression and compute its msle #
##########################################################
mieszkania_new = mieszkania.join(pd.get_dummies(mieszkania['dzielnica'])).drop('dzielnica', axis=1)
X = np.array(mieszkania_new.drop('cena', axis=1))
y = np.array(mieszkania_new['cena'])

def msle_regr(weights, bias, X=X, y=y):
  assert len(weights) == len(X[0]) and len(X) == len(y)
  y_pred = np.matmul(X, weights.T)+bias
  return msle(y, y_pred)

weights = np.zeros(shape=len(X[0]))
bias = 0

msle_regr(weights, bias)

168.71228639837256

In [7]:
X[0:5]

array([[ 104,    2,    2, 1940,    1,    0,    1,    0,    0],
       [  43,    1,    1, 1970,    1,    0,    0,    1,    0],
       [ 128,    3,    2, 1916,    1,    1,    0,    0,    0],
       [ 112,    3,    2, 1920,    1,    0,    1,    0,    0],
       [ 149,    3,    3, 1977,    0,    0,    1,    0,    0]])

Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

Hint:
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change the algorithm.
</sup></sub></sup></sub></sup></sub>

In [36]:
#############################################
# TODO: Optimize msle and compare the error #
#############################################

def weights_gradient(weights, bias, X=X, y=y):
  y_pred = np.einsum('ij,j->i', X, weights)+bias
  return -2/len(X) * np.matmul((np.log(1+y) - np.log(1+y_pred)), X)

def bias_gradient(weights, bias, X=X, y=y):
  y_pred = np.einsum('ij,j->i', X, weights)+bias
  return -2/len(X) * np.mean(np.log(1+y) - np.log(1+y_pred))

def gradients(weights, bias, X=X, y=y):
  return weights_gradient(weights, bias, X, y), bias_gradient(weights, bias, X, y)

num_epochs = 500
lr = 0.1

def train(num_epochs=100, lr=0.01, X=X, y=y):

  weights = np.zeros(shape=len(X[0]))
  bias = 0

  for i in range(num_epochs):
    g_w, g_b = gradients(weights, bias, X, y)
    weights -= lr*g_w
    bias -= lr*g_b
    p = np.matmul(X, weights)+bias
    loss = msle(y, p)

    print(f'Iter: {i:3} Loss: {loss:2.3f} Weight 0: {weights[0]:8.5f}, Bias: {bias:8.5f}')

  return weights, bias

train()

Iter:   0 Loss: 1.068 Weight 0: 21.19768, Bias:  0.00130
Iter:   1 Loss: 0.960 Weight 0: 20.26566, Bias:  0.00121
Iter:   2 Loss: 0.863 Weight 0: 19.44068, Bias:  0.00114
Iter:   3 Loss: 0.776 Weight 0: 18.72097, Bias:  0.00107
Iter:   4 Loss: 0.700 Weight 0: 18.10403, Bias:  0.00100
Iter:   5 Loss: 0.634 Weight 0: 17.58662, Bias:  0.00094
Iter:   6 Loss: 0.578 Weight 0: 17.16475, Bias:  0.00089
Iter:   7 Loss: 0.531 Weight 0: 16.83373, Bias:  0.00084
Iter:   8 Loss: 0.492 Weight 0: 16.58824, Bias:  0.00080
Iter:   9 Loss: 0.461 Weight 0: 16.42243, Bias:  0.00077
Iter:  10 Loss: 0.436 Weight 0: 16.33007, Bias:  0.00073
Iter:  11 Loss: 0.417 Weight 0: 16.30468, Bias:  0.00071
Iter:  12 Loss: 0.402 Weight 0: 16.33971, Bias:  0.00068
Iter:  13 Loss: 0.391 Weight 0: 16.42870, Bias:  0.00066
Iter:  14 Loss: 0.382 Weight 0: 16.56539, Bias:  0.00065
Iter:  15 Loss: 0.376 Weight 0: 16.74388, Bias:  0.00063
Iter:  16 Loss: 0.371 Weight 0: 16.95869, Bias:  0.00062
Iter:  17 Loss: 0.368 Weight 0:

(array([ 5.00353643e+01,  9.81493275e-01,  7.34565473e-01,  2.18315703e+02,
         6.79168867e-02, -1.56949812e-01,  1.49556420e-01,  9.24823281e-02,
         2.34612988e-02,  0.00000000e+00]),
 0.0005427511729743327)

Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code.

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

Hint:
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>

Hint 2:
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>

In [28]:
###############################################
# TODO: Implement the feature engineering part #
###############################################
mieszkania_new = mieszkania.join(pd.get_dummies(mieszkania['dzielnica']))
mieszkania_new['cena_m2'] = mieszkania_new['cena']/mieszkania_new['m2']
mieszkania_boost = mieszkania_new.join(mieszkania_new.groupby('dzielnica').mean()['cena_m2'], on='dzielnica', rsuffix='_avg').drop('dzielnica', axis=1)
mieszkania_boost['feature_1'] = mieszkania_boost['m2']*mieszkania_boost['cena_m2_avg']
mieszkania_boost.drop(['cena_m2_avg', 'cena_m2'], axis=1, inplace=True)
mieszkania_boost.head()

,m2,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena,grodziskowo,mokotowo,ochotowo,wolowo,feature_1
0,104,2,2,1940,1,780094,0,1,0,0,7.844037e+05
1,43,1,1,1970,1,346912,0,0,1,0,3.280867e+05
2,128,3,2,1916,1,523466,1,0,0,0,5.835645e+05
3,112,3,2,1920,1,830965,0,1,0,0,8.447425e+05
4,149,3,3,1977,0,1090479,0,1,0,0,1.123809e+06


In [37]:
##############################################################
# TODO: Test your solution on the training and test datasets #
##############################################################
from sklearn.preprocessing import StandardScaler

mieszkania_boost['feature_1_scaled'] = np.squeeze(StandardScaler().fit_transform([mieszkania_boost['feature_1']]))
X = np.array(mieszkania_boost.drop(['cena', 'feature_1'], axis=1))
y = np.array(mieszkania_boost['cena'])

weights = np.zeros(shape=len(X[0]))
bias = 0

print(X.shape, weights.shape)


num_epochs = 500
lr = 0.001

weights, bias = train(num_epochs, lr, X, y)

(200, 10) (10,)
Iter:   0 Loss: 2.498 Weight 0:  2.11977, Bias:  0.00013
Iter:   1 Loss: 2.197 Weight 0:  2.39535, Bias:  0.00014
Iter:   2 Loss: 1.961 Weight 0:  2.65381, Bias:  0.00016
Iter:   3 Loss: 1.772 Weight 0:  2.89789, Bias:  0.00017
Iter:   4 Loss: 1.616 Weight 0:  3.12962, Bias:  0.00018
Iter:   5 Loss: 1.486 Weight 0:  3.35055, Bias:  0.00019
Iter:   6 Loss: 1.376 Weight 0:  3.56192, Bias:  0.00020
Iter:   7 Loss: 1.281 Weight 0:  3.76474, Bias:  0.00021
Iter:   8 Loss: 1.199 Weight 0:  3.95985, Bias:  0.00022
Iter:   9 Loss: 1.127 Weight 0:  4.14793, Bias:  0.00023
Iter:  10 Loss: 1.064 Weight 0:  4.32958, Bias:  0.00024
Iter:  11 Loss: 1.008 Weight 0:  4.50533, Bias:  0.00025
Iter:  12 Loss: 0.958 Weight 0:  4.67561, Bias:  0.00026
Iter:  13 Loss: 0.913 Weight 0:  4.84081, Bias:  0.00027
Iter:  14 Loss: 0.873 Weight 0:  5.00129, Bias:  0.00027
Iter:  15 Loss: 0.836 Weight 0:  5.15735, Bias:  0.00028
Iter:  16 Loss: 0.804 Weight 0:  5.30926, Bias:  0.00029
Iter:  17 Loss:

In [38]:
mieszkania_test = pd.read_csv('mieszkania_test.csv')
mieszkania_test.head()

,m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
0,71,wolowo,2,2,1912,1,322227
1,45,mokotowo,1,1,1938,0,295878
2,38,mokotowo,1,1,1999,1,306530
3,70,ochotowo,2,2,1980,1,553641
4,136,mokotowo,3,2,1939,1,985348


In [39]:
mieszkania_new_test = mieszkania_test.join(pd.get_dummies(mieszkania_test['dzielnica']))
mieszkania_new_test['cena_m2'] = mieszkania_new_test['cena']/mieszkania_new_test['m2']
mieszkania_boost_test = mieszkania_new_test.join(mieszkania_new_test.groupby('dzielnica').mean()['cena_m2'], on='dzielnica', rsuffix='_avg').drop('dzielnica', axis=1)
mieszkania_boost_test['feature_1'] = mieszkania_boost_test['m2']*mieszkania_boost_test['cena_m2_avg']
mieszkania_boost_test.drop(['cena_m2', 'cena_m2_avg'], axis=1, inplace=True)
mieszkania_boost_test.head()

,m2,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena,grodziskowo,mokotowo,ochotowo,wolowo,feature_1
0,71,2,2,1912,1,322227,0,0,0,1,398673.241256
1,45,1,1,1938,0,295878,0,1,0,0,326664.933286
2,38,1,1,1999,1,306530,0,1,0,0,275850.388108
3,70,2,2,1980,1,553641,0,0,1,0,545855.616590
4,136,3,2,1939,1,985348,0,1,0,0,987254.020599


In [40]:
mieszkania_boost_test['feature_1_scaled'] = np.squeeze(StandardScaler().fit_transform([mieszkania_boost_test['feature_1']]))
X = np.array(mieszkania_boost_test.drop(['cena', 'feature_1'], axis=1))
y = np.array(mieszkania_boost_test['cena'])

In [42]:
def test(weights, bias, X, y):

  p = np.matmul(X, weights)+bias
  loss = msle(y, p)

  return loss

test(weights, bias, X, y)

0.42576262749454313